In [ ]:
import logging
import pandas as pd
import numpy as np
import scanpy as sc
import os
import sys

current_folder = str(globals()['_dh'][0])
ROOT_PATH = current_folder.split('src/')[0]

os.chdir(ROOT_PATH)
sys.path.append(ROOT_PATH + 'src')

import utils.basics as uf

current_folder = str(globals()['_dh'][0])
ROOT_PATH = current_folder.split('src/')[0]
os.chdir(ROOT_PATH)
# Total 80411 disease-gene triples
np.random.seed(2022)

# ! Load
state_df = uf.pickle_load(f'data/static/static_data.pkl')
trend = uf.pickle_load(f'data/dynamic/dynamic_data_trend.pkl')
risk = uf.pickle_load(f'data/dynamic/dynamic_data_risk_level.pkl')

In [ ]:
fips_to_state = dict(zip(trend['fips'], trend['state_name']))
# state_df['state_name'] = state_df.fips.map(fips_to_state)
# new_col_order = ['state_name'] + [col for col in state_df if col != 'state_name']
# state_df = state_df[new_col_order]
state_df

In [ ]:
state_df.columns

In [ ]:
trend

In [ ]:
risk

In [ ]:
trend.iloc[4727]

In [ ]:
risk.iloc[4727]

In [ ]:
risk.columns

In [ ]:
trend.columns

In [ ]:
for col in ["Week", 'Trend', 'Trend_1w', 'Trend_2w', 'Trend_3w', 'Trend_4w']:
    risk[col] = trend[col]
data = risk
data

In [ ]:
# ! Round floats
state_df[state_df.select_dtypes(include=['float64']).columns] = state_df.select_dtypes(include=['float64']).round(2)
risk[risk.select_dtypes(include=['float64']).columns] = risk.select_dtypes(include=['float64']).round(2)

In [ ]:
data.columns

In [ ]:
data.reported_cases_per_100k

In [ ]:
data.Week_start

In [ ]:
# Splitting the dataframe
df = data
train_df = df[df['Week_start'] < '2021-10-01']
val_df = df[(df['Week_start'] >= '2021-10-01') & (df['Week_start'] < '2022-10-01')]
test_df = df[df['Week_start'] >= '2022-10-01']


In [ ]:
train_df

In [ ]:
val_df

In [ ]:
test_df

In [ ]:
splits = {split: np.array(_df.index.tolist())
          for split, _df in (('train', train_df), ('val', val_df), ('test', test_df))
          }
splits

In [ ]:
label_names = data.Trend.unique()
label_info = pd.DataFrame.from_dict({"label_token": [l.upper() for l in label_names],
                                     "label_name": label_names, })
label_info

In [ ]:
merged_dynamic = data.merge(state_df, on='fips')
for target_type, target_name in (('r', 'Risk_level'), ('t', 'Trend')):
    for week in (0, 1, 2, 3, 4):
        col = target_name + (f'_{week}w' if week > 0 else '')
        merged_dynamic[f'{target_type}{week}'] = merged_dynamic[col].str.upper()
        merged_dynamic.drop(col, axis=1, inplace=True)

merged_dynamic

In [ ]:
from easydict import EasyDict

data_dict = EasyDict(
    splits=splits,
    static=state_df,
    dynamic=data,
    merged_dynamic=merged_dynamic,
    label_info=label_info,
)
uf.pickle_save(data_dict, 'data/processed.pkl')

In [ ]:
merged_dynamic.columns

In [ ]:
merged_dynamic.Dose1_Pop_Pct